# Lab 1.3.1: Parallel Reduction - SOLUTIONS

This notebook contains complete solutions to the exercises in Lab 1.3.1.

---

## 🎯 Learning Objectives Checklist

By completing this lab, you should now be able to:
- [x] Understand why parallel reduction is fundamental to GPU computing
- [x] Implement naive parallel reduction and understand its limitations
- [x] Optimize with shared memory for 10-50x speedup
- [x] Master warp shuffle operations for ultimate performance
- [x] Achieve **100x+ speedup** over CPU implementations

---

In [ ]:
import numpy as np
import time
from numba import cuda
import numba

# Verify CUDA
print(f"CUDA available: {cuda.is_available()}")

WARP_SIZE = 32
THREADS_PER_BLOCK = 256

## Solution: Parallel Max Reduction

The exercise asked to implement parallel max reduction instead of sum.

In [ ]:
@cuda.jit(device=True)
def warp_max(val):
    """
    SOLUTION: Find max within a warp using shuffle instructions.
    
    Key change: Replace += with max()
    """
    mask = 0xffffffff
    
    # Shuffle down and take max in tree pattern
    val = max(val, cuda.shfl_down_sync(mask, val, 16))
    val = max(val, cuda.shfl_down_sync(mask, val, 8))
    val = max(val, cuda.shfl_down_sync(mask, val, 4))
    val = max(val, cuda.shfl_down_sync(mask, val, 2))
    val = max(val, cuda.shfl_down_sync(mask, val, 1))
    
    return val


@cuda.jit
def parallel_max_kernel(data, partial_maxes):
    """
    SOLUTION: Find max value using parallel reduction.
    """
    # Shared memory for warp results
    warp_maxes = cuda.shared.array(shape=(THREADS_PER_BLOCK // WARP_SIZE,), dtype=numba.float32)
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    block_size = cuda.blockDim.x
    idx = bx * block_size + tx
    
    warp_id = tx // WARP_SIZE
    lane_id = tx % WARP_SIZE
    
    # Load data - use negative infinity for out-of-bounds
    val = data[idx] if idx < data.size else numba.float32(-1e38)  # -inf equivalent
    
    # Warp-level max reduction
    val = warp_max(val)
    
    # Lane 0 of each warp stores to shared memory
    if lane_id == 0:
        warp_maxes[warp_id] = val
    
    cuda.syncthreads()
    
    # First warp reduces all warp maxes
    num_warps = block_size // WARP_SIZE
    if warp_id == 0:
        val = warp_maxes[lane_id] if lane_id < num_warps else numba.float32(-1e38)
        val = warp_max(val)
        
        if lane_id == 0:
            partial_maxes[bx] = val


def gpu_max(arr: np.ndarray) -> float:
    """
    SOLUTION: Find max value using GPU.
    """
    n = arr.size
    d_data = cuda.to_device(arr)
    
    while n > 1:
        num_blocks = (n + THREADS_PER_BLOCK - 1) // THREADS_PER_BLOCK
        d_partial = cuda.device_array(num_blocks, dtype=np.float32)
        
        parallel_max_kernel[num_blocks, THREADS_PER_BLOCK](d_data, d_partial)
        
        d_data = d_partial
        n = num_blocks
    
    return d_data.copy_to_host()[0]

In [ ]:
# Test the solution
test_data = np.random.randn(1_000_000).astype(np.float32)
expected_max = np.max(test_data)

result = gpu_max(test_data)

print("Parallel Max Reduction Test")
print("="*50)
print(f"Array size: {test_data.size:,}")
print(f"Expected max (NumPy): {expected_max}")
print(f"GPU max:              {result}")
print(f"Correct: {'✅ Yes!' if abs(result - expected_max) < 1e-5 else '❌ No'}")

In [ ]:
# Benchmark
print("\nBenchmark: GPU Max vs NumPy Max")
print("="*50)

# Warm up
for _ in range(3):
    _ = gpu_max(test_data)
cuda.synchronize()

# NumPy time
start = time.perf_counter()
for _ in range(10):
    _ = np.max(test_data)
time_numpy = (time.perf_counter() - start) / 10

# GPU time
start = time.perf_counter()
for _ in range(10):
    _ = gpu_max(test_data)
    cuda.synchronize()
time_gpu = (time.perf_counter() - start) / 10

print(f"NumPy time: {time_numpy*1000:.3f} ms")
print(f"GPU time:   {time_gpu*1000:.3f} ms")
print(f"Speedup:    {time_numpy/time_gpu:.1f}x")

## Bonus Solution: Parallel Min and ArgMax

Here are additional reductions you might want:

In [ ]:
@cuda.jit(device=True)
def warp_min(val):
    """Find min within a warp."""
    mask = 0xffffffff
    val = min(val, cuda.shfl_down_sync(mask, val, 16))
    val = min(val, cuda.shfl_down_sync(mask, val, 8))
    val = min(val, cuda.shfl_down_sync(mask, val, 4))
    val = min(val, cuda.shfl_down_sync(mask, val, 2))
    val = min(val, cuda.shfl_down_sync(mask, val, 1))
    return val


@cuda.jit(device=True)
def warp_argmax(val, idx):
    """
    Find argmax within a warp.
    Returns both the max value and its original index.
    """
    mask = 0xffffffff
    
    for offset in [16, 8, 4, 2, 1]:
        other_val = cuda.shfl_down_sync(mask, val, offset)
        other_idx = cuda.shfl_down_sync(mask, idx, offset)
        
        if other_val > val:
            val = other_val
            idx = other_idx
    
    return val, idx


print("Additional reduction operations implemented:")
print("- warp_min(): Parallel minimum")
print("- warp_argmax(): Parallel argmax (returns value and index)")

## Challenge Solution: Double-Precision Reduction with Kahan Summation

For numerically stable summation:

In [ ]:
@cuda.jit
def kahan_sum_kernel(data, partial_sums, partial_comps):
    """
    Kahan summation for improved numerical accuracy.
    
    Kahan's algorithm tracks a "compensation" term that captures
    the low-order bits lost to rounding.
    
    sum = 0
    c = 0  # compensation
    for x in data:
        y = x - c        # compensated value
        t = sum + y      # new sum
        c = (t - sum) - y  # lost low-order bits
        sum = t
    """
    sdata = cuda.shared.array(shape=(THREADS_PER_BLOCK,), dtype=numba.float64)
    scomp = cuda.shared.array(shape=(THREADS_PER_BLOCK,), dtype=numba.float64)
    
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    idx = bx * cuda.blockDim.x + tx
    
    # Initialize
    if idx < data.size:
        sdata[tx] = data[idx]
    else:
        sdata[tx] = 0.0
    scomp[tx] = 0.0
    
    cuda.syncthreads()
    
    # Reduction with Kahan compensation
    stride = cuda.blockDim.x // 2
    while stride > 0:
        if tx < stride:
            # Kahan sum: sdata[tx] + sdata[tx + stride]
            y = sdata[tx + stride] - scomp[tx]
            t = sdata[tx] + y
            scomp[tx] = (t - sdata[tx]) - y
            sdata[tx] = t
        cuda.syncthreads()
        stride //= 2
    
    if tx == 0:
        partial_sums[bx] = sdata[0]
        partial_comps[bx] = scomp[0]


print("Kahan summation kernel implemented for improved numerical accuracy.")
print("\nWhen to use:")
print("- Summing many small numbers with a few large ones")
print("- When numerical precision is critical")
print("- Scientific computing applications")

## Cleanup

In [ ]:
import gc
del test_data
gc.collect()
cuda.current_context().reset()
print("✅ Cleanup complete")